In [2]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import tensorflow as tf
import urllib
from dataclasses import dataclass
import kerastuner as kt
np.random.seed(0)

/scratch/awhite38/tmp/ipykernel_44515/3486013101.py:9: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [3]:
urllib.request.urlretrieve(
    "https://github.com/whitead/dmol-book/raw/master/data/solubility.npz",
    "solubility.npz",
)
with np.load("solubility.npz") as r:
    pos_data, neg_data = r["positives"], r["negatives"]

# create labels and stich it all into one
# tensor
labels = np.concatenate(
    (
        np.ones((pos_data.shape[0], 1), dtype=pos_data.dtype),
        np.zeros((neg_data.shape[0], 1), dtype=pos_data.dtype),
    ),
    axis=0,
)
features = np.concatenate((pos_data, neg_data), axis=0)

In [4]:
@dataclass
class Config:
    vocab_size: int
    example_number: int
    batch_size: int
    buffer_size: int
        
config = Config(vocab_size=21, # include gap
                example_number=len(labels), 
                batch_size=16, 
                buffer_size=10000)

In [5]:
# we now need to shuffle before creating TF dataset
# so that our train/test/val splits are random
i = np.arange(len(labels))
np.random.shuffle(i)
labels = labels[i]
features = features[i]
data = tf.data.Dataset.from_tensor_slices((features, labels))
# now split into val, test, train and batch
N = len(data)  
split = int(0.1 * N)
test_data = data.take(split).batch(config.batch_size)
nontest = data.skip(split)
val_data, train_data = nontest.take(split).batch(config.batch_size), \
    nontest.skip(split).shuffle(config.buffer_size).batch(config.batch_size).prefetch(tf.data.experimental.AUTOTUNE)

2021-10-25 22:48:26.695131: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-25 22:48:28.289101: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38426 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0


In [9]:
def build_model(hp):
    model = tf.keras.Sequential()

    # make embedding and indicate that 0 should be treated as padding mask
    model.add(tf.keras.layers.Embedding(input_dim=config.vocab_size, 
                                        output_dim= hp.Choice('embedding_dim', [8, 16, 64]),
                                        mask_zero=True))

    # RNN layer
    model.add(tf.keras.layers.GRU( hp.Choice('rnn_units', [32, 64, 128])))
    # a dense hidden layer
    hd = hp.Choice('hidden_dim', [32, 64, 128])
    activation = hp.Choice('activation', ['relu', 'tanh', 'softplus'])
    model.add(tf.keras.layers.Dense(hd, activation=activation))
    model.add(tf.keras.layers.Dense(hd // 4, activation=activation))
    # predicting prob, so no activation
    model.add(tf.keras.layers.Dense(1))
    
    model.compile(tf.optimizers.Adam(hp.Choice('lr', [0.1, 1e-2, 1e-4])), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.keras.metrics.AUC(from_logits=True), tf.keras.metrics.BinaryAccuracy(threshold=0)])

    return model

tuner = kt.Hyperband(build_model,
                     objective=kt.Objective("val_auc", direction="max"),
                     max_epochs=25,
                     factor=3,
                     directory='/var/tmp/tuning',
                     project_name='intro_to_kt')


INFO:tensorflow:Reloading Oracle from existing project /var/tmp/tuning/intro_to_kt/oracle.json
INFO:tensorflow:Reloading Tuner from /var/tmp/tuning/intro_to_kt/tuner0.json


In [10]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_data, epochs=50, validation_data=val_data, callbacks=[stop_early])
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 13 Complete [00h 01m 46s]
val_auc: 0.5

Best val_auc So Far: 0.6387796998023987
Total elapsed time: 00h 14m 40s
INFO:tensorflow:Oracle triggered exit


In [13]:
best_hps.get('lr'), best_hps.get('rnn_units'), best_hps.get('embedding_dim'), best_hps.get('activation'), best_hps.get('hidden_dim')

(0.01, 32, 8, 'tanh', 32)